## Mini U-Net For Fingernails Segmentation

Jenia Golbstein

In [1]:
%matplotlib inline
#from utils_nails import *
from utils_nails_aug import *

Using TensorFlow backend.
/Users/osamura/.local/share/virtualenvs/Fingernails-Segmentation-LlvMCuHC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/osamura/.local/share/virtualenvs/Fingernails-Segmentation-LlvMCuHC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/osamura/.local/share/virtualenvs/Fingernails-Segmentation-LlvMCuHC/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it wil

### Initialization

In [ ]:
# initialize instance
dataset = 'nails_augment'
#dataset = 'nails'
seg = fingernailseg(dataset)
# show random example from training set
seg.plot_example(np.random.randint(seg.X_train.__len__()))
# create U-Net model
seg.create_unet()

> /Users/osamura/Documents/Fingernails-Segmentation/utils_nails_aug.py(73)__init__()
     71                             seed=47)
     72         import pdb; pdb.set_trace()
---> 73         dataset = dataset.process(True)
     74 
     75         for f in test_files:

ipdb> n
> /Users/osamura/Documents/Fingernails-Segmentation/dataloader.py(192)process()
    190         import pdb; pdb.set_trace()
    191         # data = tf.data.Dataset.from_tensor_slices((self.images, self.masks))
--> 192         data = (self.images, self.masks)
    193 
    194         # Parse images and labels

ipdb> 
ipdb> n
> /Users/osamura/Documents/Fingernails-Segmentation/dataloader.py(195)process()
    193 
    194         # Parse images and labels
--> 195         data = data.map(self._map_function, num_parallel_calls=AUTOTUNE)
    196         data = map(self._map_function, data)
    197 

ipdb> n
AttributeError: 'tuple' object has no attribute 'map'
> /Users/osamura/Documents/Fingernails-Segmentation/dataloa

In [ ]:
seg.dataset

### PreTraining

In [ ]:
if not os.path.exists('unet.h5'):
    print(" no pretrained so train start")
    seg.fit('unet.h5')
# seg.fit('unet.h5')
seg.load_model('unet.h5')

In [ ]:
# import pdb;pdb.set_trace()

mask = seg.predict()
raw = seg.X_test
plt.figure(figsize=(25,15))
for i in range(len(mask)):
    plt.subplot(3, 7, i+1)
    plt.imshow(raw[i])
    plt.imshow(mask[i,:,:,0], alpha=0.3)

### FineTuning

In [ ]:
# initialize instance
dataset = 'nails_segmentation'
seg = fingernailseg(dataset)
# show random example from training set
seg.plot_example(np.random.randint(seg.X_train.__len__()))
# create U-Net model
seg.create_unet()
seg.load_model('unet.h5')

In [ ]:
seg.finetuning_fit('unet_tuned.h5')

In [ ]:
seg.load_model('unet_tuned.h5')

# TEST

In [ ]:
mask = seg.predict()
raw = seg.X_test
plt.figure(figsize=(25,15))
for i in range(len(mask)):
    plt.subplot(3, 7, i+1)
    plt.imshow(raw[i])
    plt.imshow(mask[i,:,:,0], alpha=0.3)

In [ ]:
import cv2
img = cv2.imread("IMG_4223.JPG")
img = cv2.resize(img, (192, 160))
img = img[np.newaxis]
img.shape

In [ ]:
mask = seg.inference(img)
plt.figure(figsize=(5,5))

rand_image = np.random.randint(img.__len__())
plt.imshow(img[rand_image,:,:,:])
plt.imshow(mask[rand_image,:,:,0], alpha=0.3)
plt.title('Fingernails segmentation of test image', fontsize=15)